In [1]:
import pandas as pd
import numpy as np
import matplotlib

from calibUtils import *

In [2]:
df = pd.read_csv('./files/30-04-21_SN04calib_test5_DACmaxFFF.csv' ,delimiter=';', header=None)
df = df.rename(columns={0: 'ch', 1:'ADCmin', 2: 'DMMmin', 3:'ADCmax', 4:'DMMmax'})
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,E6DD,108.752,F6DB,116.327
1,ch1,E6BB,108.755,F6CC,116.332
2,ch2,E6B5,108.740,F6C7,116.314
3,ch3,E6AE,108.744,F6BE,116.318
4,ch4,E6C0,108.744,F6D4,116.320
5,ch5,E6B8,108.744,F6C8,116.312
6,ch6,E6AF,108.735,F6C2,116.309
7,ch7,E6B7,108.741,F6C7,116.315
8,ch8,E60D,108.745,F712,116.320
9,ch9,E6CF,108.715,F6E5,116.291


In [3]:
#convert ADCmin and ADCmax in decimal numbers
df = hex_to_dec(df)
df


,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,59101,108.752,63195,116.327
1,ch1,59067,108.755,63180,116.332
2,ch2,59061,108.740,63175,116.314
3,ch3,59054,108.744,63166,116.318
4,ch4,59072,108.744,63188,116.320
5,ch5,59064,108.744,63176,116.312
6,ch6,59055,108.735,63170,116.309
7,ch7,59063,108.741,63175,116.315
8,ch8,58893,108.745,63250,116.320
9,ch9,59087,108.715,63205,116.291


In [4]:
#data for HV calibration

data_HV = {'ADCmin': [0x02EB], 'DMMmin': [1.378], 'ADCmax': [0xFEB5], 'DMMmax': [120]}
df_HV = pd.DataFrame(data=data_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax
0,747,1.378,65205,120


In [5]:
# compute gain and offset for the Bias Voltages
# a_coeff = 1.8409e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )

df = biasV_params(df)
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV
0,ch0,59101,108.752,63195,116.327,1.005089,324.671551
1,ch1,59067,108.755,63180,116.332,1.000710,31.852184
2,ch2,59061,108.740,63175,116.314,1.000071,-3.742540
3,ch3,59054,108.744,63166,116.318,1.000557,15.799842
4,ch4,59072,108.744,63188,116.320,0.999849,-8.029567
5,ch5,59064,108.744,63176,116.312,0.999765,-21.006342
6,ch6,59055,108.735,63170,116.309,0.999828,-21.383021
7,ch7,59063,108.741,63175,116.315,1.000557,26.428571
8,ch8,58893,108.745,63250,116.320,0.944419,-3655.114191
9,ch9,59087,108.715,63205,116.291,0.999363,-5.973865


In [6]:
#Compute gain and offset for the HV

# a_coeff = 1.8535e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )


df_HV = HV_params(df_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV
0,747,1.378,65205,120,0.992878,-1.79132


In [7]:
#convert to ADC the bias voltages 
# from page 13, ADC calib = 108.7677 + 1.8525e-3 * ADC <-- it's too high!
#df = convertADC_BiasV(df)
#df

In [8]:
#df_HV = convertADC(df_HV)  
#df_HV

In [9]:
# Gain biasV = math.ceil(Gain BiasV* 32768)
# Offset biasV = round(offset BiasV)  

df = code_offsetandgain(df,'biasV')
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV
0,ch0,59101,108.752,63195,116.327,1.005089,324.671551,32935,325
1,ch1,59067,108.755,63180,116.332,1.000710,31.852184,32791,32
2,ch2,59061,108.740,63175,116.314,1.000071,-3.742540,32770,-4
3,ch3,59054,108.744,63166,116.318,1.000557,15.799842,32786,16
4,ch4,59072,108.744,63188,116.320,0.999849,-8.029567,32763,-8
5,ch5,59064,108.744,63176,116.312,0.999765,-21.006342,32760,-21
6,ch6,59055,108.735,63170,116.309,0.999828,-21.383021,32762,-21
7,ch7,59063,108.741,63175,116.315,1.000557,26.428571,32786,26
8,ch8,58893,108.745,63250,116.320,0.944419,-3655.114191,30947,-3655
9,ch9,59087,108.715,63205,116.291,0.999363,-5.973865,32747,-6


In [10]:
# Gain HV = math.ceil(Gain HV* 32768)
# Offset HV = round(offset HV)  

df_HV = code_offsetandgain(df_HV,'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV
0,747,1.378,65205,120,0.992878,-1.79132,32535,-2


In [11]:
# convert following prescriptions
 # GainUsig BiasV   = hex(Gain_Usig BiasV) 
 # Offset_Sig BiasV = tohextwocompl(Offset_Sig BiasV ,15) 

df = convert_to_exadec(df, 'biasV')
df

,ch,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV
0,ch0,59101,108.752,63195,116.327,1.005089,324.671551,0x80a7,0x145
1,ch1,59067,108.755,63180,116.332,1.000710,31.852184,0x8017,0x20
2,ch2,59061,108.740,63175,116.314,1.000071,-3.742540,0x8002,0x7ffc
3,ch3,59054,108.744,63166,116.318,1.000557,15.799842,0x8012,0x10
4,ch4,59072,108.744,63188,116.320,0.999849,-8.029567,0x7ffb,0x7ff8
5,ch5,59064,108.744,63176,116.312,0.999765,-21.006342,0x7ff8,0x7feb
6,ch6,59055,108.735,63170,116.309,0.999828,-21.383021,0x7ffa,0x7feb
7,ch7,59063,108.741,63175,116.315,1.000557,26.428571,0x8012,0x1a
8,ch8,58893,108.745,63250,116.320,0.944419,-3655.114191,0x78e3,0x71b9
9,ch9,59087,108.715,63205,116.291,0.999363,-5.973865,0x7feb,0x7ffa


In [12]:
# convert following prescriptions
# GainUsig BiasV   = hex(Gain_Usig HV) 
# Offset_Sig BiasV = tohextwocompl(Offset_Sig HV ,15) 

df_HV = convert_to_exadec(df_HV, 'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV
0,747,1.378,65205,120,0.992878,-1.79132,0x7f17,0x7ffe


In [13]:
df_calib_biasV = df[['Gain_Usig BiasV','Offset_Sig BiasV'] ].copy()
df_calib_biasV.to_csv('calibDB/calib_biasV_SN04_v5.csv')

In [14]:
df_calib_HV = df_HV[['Gain_Usig HV','Offset_Sig HV'] ].copy()
df_calib_HV.to_csv('calibDB/calib_HV_SN04_v5.csv')

In [15]:
df_calib_biasV

,Gain_Usig BiasV,Offset_Sig BiasV
0,0x80a7,0x145
1,0x8017,0x20
2,0x8002,0x7ffc
3,0x8012,0x10
4,0x7ffb,0x7ff8
5,0x7ff8,0x7feb
6,0x7ffa,0x7feb
7,0x8012,0x1a
8,0x78e3,0x71b9
9,0x7feb,0x7ffa


In [16]:
df_calib_HV

,Gain_Usig HV,Offset_Sig HV
0,0x7f17,0x7ffe
